In [1]:
import pandas as pd
import torchvision
from shared import InputDataset
import torch
from tqdm import tqdm
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64

In [2]:
df = pd.read_hdf("allfeatures.hdf")
df = df.fillna(df.mean())
df = df.iloc[::20]
df

,trade_in_1h_spot_ADAUSDT_close_price,daytime,spot_ADAUSDT_open_price_ewm_mean,spot_ADAUSDT_open_price_ewm_std,spot_ADAUSDT_open_price_ewm_cov,shift_0_spot_ADAUSDT_open_price_mean_120,shift_0_spot_ADAUSDT_open_price_skew_120,shift_0_spot_ADAUSDT_open_price_kurt_120,shift_0_spot_ADAUSDT_open_price_std_120,shift_240_spot_ADAUSDT_open_price_mean_120,...,shift_0_spot_ADAUSDT_taker_buy_quote_asset_volume_kurt_60,shift_0_spot_ADAUSDT_taker_buy_quote_asset_volume_std_60,shift_240_spot_ADAUSDT_taker_buy_quote_asset_volume_mean_60,shift_240_spot_ADAUSDT_taker_buy_quote_asset_volume_skew_60,shift_240_spot_ADAUSDT_taker_buy_quote_asset_volume_kurt_60,shift_240_spot_ADAUSDT_taker_buy_quote_asset_volume_std_60,shift_480_spot_ADAUSDT_taker_buy_quote_asset_volume_mean_60,shift_480_spot_ADAUSDT_taker_buy_quote_asset_volume_skew_60,shift_480_spot_ADAUSDT_taker_buy_quote_asset_volume_kurt_60,shift_480_spot_ADAUSDT_taker_buy_quote_asset_volume_std_60
2021-01-04 11:20:59.999000+00:00,0.036722,-0.074531,-1.909687,-0.648146,-0.176138,-1.913100,-1.131116,0.623300,-0.626334,-1.871625,...,-0.846745,-0.630959,-0.168640,0.030476,0.021407,-0.394865,-0.661803,-0.426065,-0.297632,-0.726861
2021-01-04 11:40:59.999000+00:00,0.036938,-0.074531,-1.910256,-0.740519,-0.181543,-1.913065,-1.616506,1.056645,-0.646207,-1.874002,...,2.199858,-0.396540,-0.047330,-0.447464,-0.473253,-0.205394,-0.593374,-0.792412,-0.748585,-0.653821
2021-01-04 12:00:59.999000+00:00,0.021161,0.069946,-1.905108,-0.470303,-0.161243,-1.912016,-1.446041,1.229738,-0.619081,-1.877475,...,1.639968,-0.369021,-0.078351,-0.357117,-0.282919,-0.287197,-0.529596,-0.549676,-0.520019,-0.601402
2021-01-04 12:20:59.999000+00:00,0.013838,0.069946,-1.902825,-0.694666,-0.179059,-1.910077,-0.905540,0.416597,-0.648592,-1.881148,...,2.226474,-0.384426,-0.047025,-0.491028,-0.320946,-0.292472,-0.346014,-0.845703,-0.754393,-0.438681
2021-01-04 12:40:59.999000+00:00,0.011350,0.069946,-1.899724,-0.763532,-0.182642,-1.906968,0.170050,-0.562061,-0.711798,-1.886245,...,-0.866335,-0.610321,-0.119398,-1.151383,-0.767618,-0.438395,-0.287920,-0.932421,-0.825982,-0.426275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-29 19:33:59.999000+00:00,0.001496,1.081289,0.810568,-0.658186,-0.176803,0.820511,0.031403,-0.895658,-0.437345,0.807455,...,0.763435,-0.550191,-0.037214,-0.718836,-0.686781,-0.066073,-0.639102,-0.673427,-0.630258,-0.683486
2021-10-29 19:53:59.999000+00:00,0.002985,1.081289,0.815557,-0.483507,-0.162552,0.817700,0.351614,-0.854022,-0.541512,0.812459,...,-0.104784,-0.465197,-0.036275,-1.037761,-0.866607,-0.110921,-0.526061,-0.636592,-0.702661,-0.494780
2021-10-29 20:13:59.999000+00:00,-0.001986,1.225766,0.821810,-0.471555,-0.161368,0.817108,0.471610,-0.551048,-0.578451,0.818685,...,-0.275976,-0.516271,-0.046592,-0.912186,-0.832715,-0.075191,-0.493173,-0.805261,-0.780498,-0.492350
2021-10-29 20:33:59.999000+00:00,0.000995,1.225766,0.820460,-0.610927,-0.173509,0.816629,0.372661,-0.450580,-0.615412,0.824923,...,-0.358129,-0.512971,-0.110866,-0.538919,-0.620419,-0.148047,-0.349651,0.136156,-0.052606,-0.235194


In [3]:
validation_size = int(len(df) / 5)
dataset_valid = InputDataset(df=df.iloc[-validation_size:], device=DEVICE)

In [4]:
model = torchvision.models.resnet101(pretrained=True)
model.load_state_dict(torch.load("best_model"))
model.eval()
model.to(DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
validation_size = int(len(df) / 5)
dataset_train = InputDataset(df=df.iloc[:-validation_size].sample(frac=1), device=DEVICE, real_profit=True)
dataset_valid = InputDataset(df=df.iloc[-validation_size:], device=DEVICE, real_profit=True)


train_loader1 = torch.utils.data.DataLoader(
    dataset=dataset_train, batch_size=BATCH_SIZE, drop_last=True
)
validation_loader1 = torch.utils.data.DataLoader(
    dataset=dataset_valid, batch_size=BATCH_SIZE, drop_last=True
)

dataloaders_dict = {'train':train_loader1, 'val':validation_loader1}

In [6]:
predictions = []
price_changes = []
for inputs, label in tqdm(validation_loader1):
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for one in preds:
            predictions.append(0 if 0 < one else 1)
        for one in label:
            price_changes.append(one)
        
        
assert len(predictions) == len(price_changes)

  0%|          | 0/66 [00:00<?, ?it/s]/home/vaclavmatejka/anaconda3/envs/dj/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 66/66 [00:13<00:00,  4.91it/s]


In [7]:
price_changes

[tensor(0.0145, device='cuda:0'),
 tensor(0.0160, device='cuda:0'),
 tensor(0.0193, device='cuda:0'),
 tensor(0.0043, device='cuda:0'),
 tensor(0.0043, device='cuda:0'),
 tensor(-0.0064, device='cuda:0'),
 tensor(0.0007, device='cuda:0'),
 tensor(0.0042, device='cuda:0'),
 tensor(0.0117, device='cuda:0'),
 tensor(0.0139, device='cuda:0'),
 tensor(0.0079, device='cuda:0'),
 tensor(0.0050, device='cuda:0'),
 tensor(0.0122, device='cuda:0'),
 tensor(-0.0004, device='cuda:0'),
 tensor(0.0068, device='cuda:0'),
 tensor(-0.0032, device='cuda:0'),
 tensor(-0.0212, device='cuda:0'),
 tensor(-0.0323, device='cuda:0'),
 tensor(-0.0218, device='cuda:0'),
 tensor(-0.0170, device='cuda:0'),
 tensor(-0.0128, device='cuda:0'),
 tensor(-0.0145, device='cuda:0'),
 tensor(-0.0054, device='cuda:0'),
 tensor(-0.0142, device='cuda:0'),
 tensor(-0.0175, device='cuda:0'),
 tensor(-0.0094, device='cuda:0'),
 tensor(-0.0029, device='cuda:0'),
 tensor(-0.0058, device='cuda:0'),
 tensor(0.0061, device='cuda:0'),

In [8]:
len(price_changes)

4224

In [9]:
money = 1000
profits = 0
losses = 0
fee_rate = 0.00018
#fee_rate = 0.0
for i in range(len(price_changes)):
    price_change = price_changes[i]
    prediction = predictions[i]
    change = money * (abs(price_changes[i]))
    profit = -1
    if price_change > 0 and prediction ==1:
        profit = 1
    if price_change < 0 and prediction == 0:
        profit = 1
    
    change = change * profit
    
    fee = money * fee_rate
    money = money - fee
    money = money + change
    
    
    
    if profit == 1:
        profits +=1
    elif profit == -1:
        losses += 1
    else:
        raise Exception(i)

money
    
    
    
    

tensor(5376.8701, device='cuda:0')

In [10]:
losses


2069

In [11]:
profits

2155